# Logistic Regression - Optimized Model

Optimize logistic regression using recursive feature elimination to select the most important features.

## 1. Setup

In [2]:
# Core data science libraries
import pandas as pd
from test_case_fairmoney.utils.constants import PATH_DATA_COMPRESSED, PATH_MODELS
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score
from test_case_fairmoney.utils.lr_recursive_elimination import (
    recursive_feature_elimination,
)
from test_case_fairmoney.utils.model_validation import plot_AUC, plot_PR

## 2. Load Data

In [3]:
df_train = pd.read_parquet(PATH_DATA_COMPRESSED / "df_train.parquet")
df_val = pd.read_parquet(PATH_DATA_COMPRESSED / "df_val.parquet")
df_test = pd.read_parquet(PATH_DATA_COMPRESSED / "df_test.parquet")

df_train_woe = pd.read_parquet(PATH_DATA_COMPRESSED / "df_train_woe.parquet")
df_val_woe = pd.read_parquet(PATH_DATA_COMPRESSED / "df_val_woe.parquet")
df_test_woe = pd.read_parquet(PATH_DATA_COMPRESSED / "df_test_woe.parquet")

## 3. Initial Feature Set

In [4]:
target = "default"
selected_lr_features = [
    "checking_balance",
    "months_loan_duration",
    "savings_balance",
    "amount",
    "employment_months",
    "installment_rate",
    "residence_months",
    "existing_credits",
    "credit_history",
    "is_checking_negative",
    "installment_plan",
    "purpose",
    "property",
    "housing",
    "personal_status",
    "job",
    "telephone_country_code",
    "gender",
]

## 4. Recursive Feature Elimination

In [10]:
# Perform recursive feature elimination
feat_drop, elimination_history = recursive_feature_elimination(
    df_train_woe[selected_lr_features],
    df_train[target],
    df_val_woe[selected_lr_features],
    df_val[target],
    df_test_woe[selected_lr_features],
    df_test[target],
    max_iterations=10,
)

# Display elimination history
if elimination_history:
    elimination_df = pd.DataFrame(elimination_history)
    print("\nFeature Elimination History:")
    print(elimination_df)

Starting Recursive Feature Elimination...
Initial features: 18

=== Iteration 1 ===
Features dropped so far: 0
Testing removal from 18 remaining features...
  Testing removal of feature (1/18): checking_balance
    Train AUC: 0.8456, Val AUC: 0.7143, Test AUC: 0.7388
  Testing removal of feature (2/18): months_loan_duration
    Train AUC: 0.8481, Val AUC: 0.6905, Test AUC: 0.7369
  Testing removal of feature (3/18): savings_balance
    Train AUC: 0.8506, Val AUC: 0.7000, Test AUC: 0.7529
  Testing removal of feature (4/18): amount
    Train AUC: 0.8442, Val AUC: 0.6977, Test AUC: 0.7265
  Testing removal of feature (5/18): employment_months
    Train AUC: 0.8497, Val AUC: 0.6945, Test AUC: 0.7426
  Testing removal of feature (6/18): installment_rate
    Train AUC: 0.8497, Val AUC: 0.7070, Test AUC: 0.7365
  Testing removal of feature (7/18): residence_months
    Train AUC: 0.8544, Val AUC: 0.7073, Test AUC: 0.7523
  Testing removal of feature (8/18): existing_credits
    Train AUC: 0.8

In [ ]:
# Dropped features
print("Dropped features:", elimination_df["feature_dropped"].tolist())

['checking_balance',
 'installment_plan',
 'installment_rate',
 'residence_months',
 'job',
 'telephone_country_code',
 'existing_credits',
 'gender',
 'savings_balance',
 'personal_status']

## 5. Train Final Model

In [5]:
selected_lr_features_final = [
    "months_loan_duration",
    "amount",
    "employment_months",
    "credit_history",
    "purpose",
    "property",
    "housing",
    "is_checking_negative",
]
model = LogisticRegression(penalty="l2", C=0.1, max_iter=1000, random_state=42)
model.fit(df_train_woe[selected_lr_features_final], df_train[target])

# Generate predictions
pred_train = model.predict_proba(df_train_woe[selected_lr_features_final])[:, 1]
pred_val = model.predict_proba(df_val_woe[selected_lr_features_final])[:, 1]
pred_test = model.predict_proba(df_test_woe[selected_lr_features_final])[:, 1]

# Calculate and display performance metrics
print("\n" + "=" * 60)
print("FINAL MODEL PERFORMANCE METRICS")
print("=" * 60)
print(f"Train AUC: {roc_auc_score(df_train[target], pred_train):.4f}")
print(f"Validation AUC: {roc_auc_score(df_val[target], pred_val):.4f}")
print(f"Test AUC: {roc_auc_score(df_test[target], pred_test):.4f}")
print(f"Train PR-AUC: {average_precision_score(df_train[target], pred_train):.4f}")
print(f"Validation PR-AUC: {average_precision_score(df_val[target], pred_val):.4f}")
print(f"Test PR-AUC: {average_precision_score(df_test[target], pred_test):.4f}")


FINAL MODEL PERFORMANCE METRICS
Train AUC: 0.8056
Validation AUC: 0.7205
Test AUC: 0.7725
Train PR-AUC: 0.6059
Validation PR-AUC: 0.4989
Test PR-AUC: 0.6282


## 6. Save Model

In [ ]:
import joblib

joblib.dump(model, PATH_MODELS / "logistic_regression_optimized.joblib")
print("✅ Model saved")

## 7. ROC-AUC Curves

In [14]:
print("ROC-AUC Analysis and Visualization")
plot_AUC(
    df_train[target],  # Train true labels
    pred_train,  # Train predictions
    df_val[target],  # Validation true labels
    pred_val,  # Validation predictions
    df_test[target],  # Test true labels
    pred_test,  # Test predictions
)

ROC-AUC Analysis and Visualization


## 8. Precision-Recall Curves

In [15]:
# Precision-Recall curve analysis and visualization
print("Precision-Recall Analysis and Visualization")
plot_PR(
    df_train[target],  # Train true labels
    pred_train,  # Train predictions
    df_test[target],  # Test true labels
    pred_test,  # Test predictions
    df_val[target],  # Validation true labels
    pred_val,  # Validation predictions
)

Precision-Recall Analysis and Visualization


## Summary

✅ **Feature Selection**: Reduced from 18 to 8 features  
✅ **Model Saved**: Ready for deployment  
⏭️ **Next**: Compare with CatBoost performance